# HW2

In [2]:
import copy
import math
from tdTTT import TicTacToe3D
from typing import Tuple, Optional, Callable

### Player 1: Alpha-beta forward pruning with heuristic 1

In [ ]:
#Heuristics for Player1 that computes a score only for the longest sequence
def player1_score(current_state: TicTacToe3D, current_player: str) -> int:
    return 10 * longest_sequence(current_state, current_player)

#Helper function to find the longest sequence of the given player
def longest_sequence(current_state: TicTacToe3D, current_player: str) -> int:
    #Initialize the size of the board and the returning value
    size = current_state.size  
    longest_count = 0
    #These loops iterate over each cell in three dimensions
    for i in range(size):
        for j in range(size):
            for k in range(size):
                count = 0
                #Iterates over horizontal lines
                for x in range(i, size):
                    #Count the encountered sequence
                    if current_state.board[k][j][x] == current_player:
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                #Iterates over vertical lines
                for y in range(j, size):
                    #Count the encountered sequence
                    if current_state.board[k][y][i] == current_player:
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                #Iterates over the third dimension
                for z in range(k, size):
                    #Count the encountered sequence
                    if current_state.board[z][j][i] == current_player:
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                #Iterates over xy plane
                for m in range(min(size - i, size - j)):
                    #Count the encountered sequence
                    if current_state.board[k][j + m][i + m] == current_player:
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                #Iterates over xz plane
                for m in range(min(size - i, size - k)):
                    #Count the encountered sequence
                    if current_state.board[k + m][j][i + m] == current_player:
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                #Iterates over yz plane
                for m in range(min(size - j, size - k)):
                    #Count the encountered sequence
                    if current_state.board[k + m][j + m][i] == current_player:
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                #Itertes over zyx plane
                for m in range(min(size - i, size - j, size - k)):
                    #Count the encountered sequence
                    if current_state.board[k + m][j + m][i + m] == current_player:
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                #The last three loops explore the sequences in different directions(tilted along axises)
                for m in range(min(size - i, j + 1, k + 1)):
                    #Count the encountered sequence
                    if current_state.board[k - m][j - m][i + m] == current_player: #Tilted along z and y plane
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                for m in range(min(size - i, j + 1, size - k)):
                    #Count the encountered sequence
                    if current_state.board[k + m][j - m][i + m] == current_player: #Tilted along y plane
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

                count = 0
                for m in range(min(size - i, size - j, k + 1)):
                    #Count the encountered sequence
                    if current_state.board[k - m][j + m][i + m] == current_player: #Tilted along z plane
                        count += 1
                    else:
                        break #Break, if the sequence stopped
                longest_count = max(longest_count, count) #After the loop, find the longest one

    return longest_count    

### Player 2: Alpha-beta forward pruning with heuristic 2

In [ ]:
#Heuristics for Player2 that focuses on the center of the board
def player2_score(current_state: TicTacToe3D, current_player: str) -> int:
    if current_player == 'X':
        other_player = 'O' #Determine the opponent
    else:   
        other_player = 'X'
    #Find longest sequences for Player2 and Player1
    current_player_sequence = longest_sequence(current_state, current_player)
    other_player_sequence = longest_sequence(current_state, other_player)
    #Count the number of occupied center cells
    center_cells_count = center_cells(current_state, current_player)
    #The function focuses on all aspects: make its own sequences, prioritize center spots, and block the opponent
    return (5 * current_player_sequence) + (4 * center_cells_count) - (6 * other_player_sequence)

#Helper function to find the number of occupied center cells
def center_cells(current_state: TicTacToe3D, current_player: str) -> int:
    cells = 0
    #The dimensions of the board are 5x5x5(as stated in the instruction slides), so it makes sense to focus on the middle indices(1 to 3)
    for i in range(1, 4):
        for j in range(1, 4): #Loops iterate over three dimensions
            for k in range(1, 4):
                if current_state.board[i][j][k] == current_player:
                    cells += 1 #If the player occupies a cell, increment the variable
    return cells

#Helper function to copy the board and make a move
def copy_board(state: TicTacToe3D, move: Tuple[int, int, int], current_player: str) -> TicTacToe3D:
    new_state = copy.deepcopy(state) #Create a copy
    x, y, z = move #Obtain the coordinates of the given move
    new_state.board[z][y][x] = current_player #Make a move
    winner = new_state.check_winner() #Check if there is a winner
    if winner:
        new_state.winner = winner
    return new_state

#Alpha-beta search with forward pruning
#maximize_player - the player for who we are maximizing the score, chosen_heuristic - player1_score or player2_score. 
#The function returns the best score and the best move(or None)
def search(current_state: TicTacToe3D, current_player: str, maximize_player: str, alpha: float, beta: float, depth: int, limit: int, chosen_heuristic: Callable[[TicTacToe3D, str], int]) -> Tuple[int, Optional[Tuple[int, int, int]]]:
    #The base case of the recursive function: stop when the terminal state or depth has been reached
    if current_state.winner is not None or depth == 0:
        return chosen_heuristic(current_state, maximize_player), None #Return the best score

    legal_moves = current_state.get_legal_moves() #Find all legal moves for the current state
    possible_moves = [] #Create a new list
    for move in legal_moves: #Iterate over all legal moves
        new_state = copy_board(current_state, move, current_player) #Make a move on the copied board and return a new state
        score = chosen_heuristic(new_state, maximize_player) #Evaluate the score
        possible_moves.append((score, move)) #Append the evaluated move into the list

    #Sort the list either in ascending or descending order(for maximizing/minimizing)
    sorted_possible_moves = sorted(possible_moves, reverse=(current_player == maximize_player))
    updated_moves = []
    for i in sorted_possible_moves[:limit]: #Based on the given limit the loop updates moves 
        move = i[1]
        updated_moves.append(move)

    if current_player == 'X':
        next_player = 'O' #Determine the next player
    else:
        next_player = 'X'
    best_move = None

    if current_player == maximize_player:  #If the player tries to maximize
        max_value = -math.inf #We start with the lowest possible value(negative infinity)
        for move in updated_moves: #Iterate over the list with updated moves
            next_state = copy_board(current_state, move, current_player) #Make a move for each possible one
            #Recursively search for the result using the new state
            result = search(next_state, next_player, maximize_player, alpha, beta, depth - 1, limit, chosen_heuristic) 
            score = result[0]
            if score > max_value:
                max_value = score #Update if the better score is found
                best_move = move #Update if the better move is found
            alpha = max(alpha, score)
            if beta <= alpha: #Prune the move if we know that minimizer won't choose it
                break
        return max_value, best_move
    else:  #If the player tries to minimize
        min_value = math.inf #We start with the highest possible value(infinity)
        for move in updated_moves: #Iterate over the list with updated moves
            next_state = copy_board(current_state, move, current_player) #Make a move for each possible one
            #Recursively search for the result using the new state
            result = search(next_state, next_player, maximize_player, alpha, beta, depth - 1, limit, chosen_heuristic) 
            score = result[0]
            if score < min_value:
                min_value = score #Update if the better score is found
                best_move = move #Update if the better move is found
            beta = min(beta, score)
            if beta <= alpha: #Prune the move if we know that maximizer won't choose it
                break
        return min_value, best_move

### Competition

The two players face each other 100 times, switching who starts each time. 

In [ ]:
player1 = 0
player2 = 0
draws = 0

for i in range(100): #Run 100 games
    game = TicTacToe3D(5) #Create new 5x5x5 board
    if i % 2 == 0: #Change the player who moves first
        current_player = 'X'
    else:
        current_player = 'O'

    while game.get_legal_moves() and not game.winner: #Iterate until there is a winner or draw
        if current_player == 'X': #Use the first heuristic
            result = search(game, 'X', 'X', -math.inf, math.inf, 2, 7, player1_score)
            move = result[1]
        else: #Use the second heuristic
            result = search(game, 'O', 'O', -math.inf, math.inf, 2, 7, player2_score)
            move = result[1]
        if move:
            x, y, z = move
            game.board[z][y][x] = current_player #Now, make a real move on the board
            game.winner = game.check_winner() #Find a winner
        if current_player == 'X':
            current_player = 'O' #Determine the next player
        else:
            current_player = 'X'
    #Increment the winner's score
    if game.winner == 'X': 
        player1 += 1
    elif game.winner == 'O':
        player2 += 1
    else:
        draws += 1
#Print the results
print(f"{100} Games Statistics:")
print(f"X Wins: {player1}")
print(f"O Wins: {player2}")
print(f"Draws: {draws}")

"""

Analysis.

The first heuristic searches for the longest sequence of its moves. The strong side of it is that it focuses very hard on  
attacking the opponent. However, it doesn't have any defensive mechanism to block the opponent's winning lines.  
The second heuristic is more balanced than the first one. It blocks the other player and controls the center of the board at  
the same time. It is hard to defeat, so it also provides good competition for Player1. However, it can miss some of the  
winning lines due to the focus on defense.  
In the competition part, I used the depth of 2 and the limit of 7. Since we had to run 100 games, 2 and 7 are reasonable choices  
to trade off between speed and quality of choices. My results are the following for the 5x5x5 board:  

X Wins: 50  
O Wins: 50  
Draws: 0  

I think that I got these results since I chose two different strategies for my evaluation function. One attacks, while the other blocks.  
However, initially, I supposed that Player2 would win more games than Player1, but due to the depth of only 2, there might not be enough  
opportunities for it to look that far ahead.
Also, it is important to mention that first-move advantage exists. Therefore, the similar win rates between the two heuristics may not
fully reflect their strengths, as the order played a significant role. There is still room for improvement, such as
increasing the search depth and designing stronger heuristics with threat detection to help reduce the impact of the first-move advantage.

"""